In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.vec_env import DummyVecEnv

from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])


import warnings
warnings.filterwarnings('ignore')
import itertools

In [2]:
# ETFs
TICKERS = ['XLP', 'XLY', 'XLI', 'XLE', 'XLK', 'IYZ', 'XRT', 'XLV', 'XLU', 'VTI']

# Mutual Funds
# TICKERS = ['VCSAX', 'FSCPX', 'VINAX', 'FSENX', 'VITAX', 'FSDCX', 'FSRPX', 'VGHCX', 'VUIAX', 'VEXAX']

# Futures
# TICKERS = ['SPSU', 'SPSD', 'SPSI', 'SPEN', 'SPTL', 'SPTS', 'SPSD', 'SPHC', 'SPUT', 'ES']

START_DATE = '1980-01-01'
END_DATE = '2024-12-31'
TRAIN_END_DATE = '2020-12-31'
TEST_START_DATE = '2021-01-01'

print('Downloading data...')
stock_data = YahooDownloader(
    start_date=START_DATE,
    end_date=END_DATE,
    ticker_list=TICKERS,
).fetch_data()

stock_data.head()

YF deprecation warning: set proxy via new config function: yf.set_config(proxy=proxy)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (62600, 8)


Price,date,close,high,low,open,volume,tic,day
0,1998-12-22,11.746048,11.809156,11.706605,11.769714,15200,XLE,1
1,1998-12-22,14.582221,14.582221,14.533287,14.533287,600,XLI,1
2,1998-12-22,23.943207,24.281750,23.744750,24.211707,300500,XLK,1
3,1998-12-22,14.274880,14.291714,13.938209,13.938209,150300,XLP,1
4,1998-12-22,11.913817,12.082321,11.913817,12.082321,7900,XLU,1


In [3]:
stock_data["date"] = pd.to_datetime(stock_data["date"])

In [4]:
df = stock_data.copy()
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

print(f"Date column type before: {stock_data['date'].dtype}")
print(f"Date column type after: {df['date'].dtype}")

fe = FeatureEngineer(
    use_technical_indicator=True,
    tech_indicator_list=INDICATORS,
    use_vix=True,
    use_turbulence=True,
    user_defined_feature=False,
)

print("Preprocessing data with FeatureEngineer...")
processed_data = fe.preprocess_data(df)

# Convert date to datetime after processing
processed_data["date"] = pd.to_datetime(processed_data["date"])
processed_data = processed_data.dropna().reset_index(drop=True)

print(f"Processed data shape: {processed_data.shape}")
print(f"Final date column type: {processed_data['date'].dtype}")
print(f"Features: {processed_data.columns.tolist()}")

processed_data.head()

Date column type before: datetime64[ns]
Date column type after: object
Preprocessing data with FeatureEngineer...


[*********************100%***********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (6546, 8)


Successfully added vix
Successfully added turbulence index
Processed data shape: (45822, 18)
Final date column type: datetime64[ns]
Features: ['date', 'close', 'high', 'low', 'open', 'volume', 'tic', 'day', 'macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma', 'vix', 'turbulence']


,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,1998-12-22,11.746048,11.809156,11.706605,11.769714,15200,XLE,1,0.0,12.214163,11.52248,100.0,66.666667,100.0,11.746048,11.746048,22.780001,0.0
1,1998-12-22,14.582221,14.582221,14.533287,14.533287,600,XLI,1,0.0,12.214163,11.52248,100.0,66.666667,100.0,14.582221,14.582221,22.780001,0.0
2,1998-12-22,23.943207,24.281750,23.744750,24.211707,300500,XLK,1,0.0,12.214163,11.52248,100.0,66.666667,100.0,23.943207,23.943207,22.780001,0.0
3,1998-12-22,14.274880,14.291714,13.938209,13.938209,150300,XLP,1,0.0,12.214163,11.52248,100.0,66.666667,100.0,14.274880,14.274880,22.780001,0.0
4,1998-12-22,11.913817,12.082321,11.913817,12.082321,7900,XLU,1,0.0,12.214163,11.52248,100.0,66.666667,100.0,11.913817,11.913817,22.780001,0.0


In [5]:
processed_data = processed_data.sort_values(["date", "tic"], ignore_index=True)
processed_data.index = processed_data.date.factorize()[0]

cov_list = []
return_list = []

lookback = 252
for i in range(lookback, len(processed_data.index.unique())):
    data_lookback = processed_data.iloc[i - lookback : i]
    price_lookback = data_lookback.pivot(index="date", columns="tic", values="close")
    return_lookback = price_lookback.pct_change().dropna()
    return_list.append(return_lookback)

    cov = return_lookback.cov()
    cov_list.append(cov)

df_cov = pd.DataFrame(
    {
        "date": processed_data.date.unique()[lookback:],
        "cov_list": cov_list,
        "return_list": return_list,
    }
)
processed_data = processed_data.merge(df_cov, on="date")
processed_data = processed_data.sort_values(["date", "tic"]).reset_index(drop=True)

processed_data.head()

,date,close,high,low,open,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence,cov_list,return_list
0,1999-12-22,13.516113,13.628347,13.500080,13.548180,605300,XLE,2,-0.103611,14.166825,13.373424,45.574863,-104.982141,22.226879,13.977271,13.829792,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
1,1999-12-22,17.689075,17.788008,17.629716,17.738542,511700,XLI,2,0.010758,17.973611,17.176689,49.821596,11.634124,7.185993,17.667049,17.639549,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
2,1999-12-22,39.645416,40.112382,39.120079,39.972292,307700,XLK,2,1.361657,39.794943,34.553056,69.913430,160.096859,38.563402,36.267857,33.651680,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
3,1999-12-22,12.483125,12.576727,12.321448,12.363995,750600,XLP,2,-0.275549,14.056472,11.978243,42.296755,-110.385131,31.928089,13.197083,13.134717,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...
4,1999-12-22,11.768037,11.889819,11.742398,11.857771,84600,XLU,2,-0.033414,12.055391,11.618068,47.905714,-59.163569,6.749451,11.878774,11.891841,22.43,0.0,tic XLE XLI XLK XLP ...,tic XLE XLI XLK ...


In [6]:
train = data_split(processed_data, TRAIN_START_DATE, end=TRAIN_END_DATE)
trade = data_split(processed_data, TRAIN_START_DATE, end=TRADE_END_DATE)

In [7]:
from finrl.meta.env_portfolio_allocation.env_portfolio import StockPortfolioEnv

In [8]:
stock_dimensions = len(train.tic.unique())
state_space = stock_dimensions
print(f'Stock dimensions: {stock_dimensions}, State Space: {state_space}')

Stock dimensions: 7, State Space: 7


In [9]:
env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "transaction_cost_pct": 0.005,
    "state_space": state_space,
    "stock_dim": stock_dimensions,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimensions,
    "reward_scaling": 1e-4,
}

e_train_gym = StockPortfolioEnv(df=train, **env_kwargs)
e_trade_gym = StockPortfolioEnv(df=trade, **env_kwargs)

In [10]:
models_to_train = {
    "PPO": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            "learning_rate": 0.0003,
            "n_steps": 2048,
            "batch_size": 64,
            "n_epochs": 10,
            "gamma": 0.99,
            "gae_lambda": 0.95,
            "clip_range": 0.2,
            "vf_coef": 0.5,
            "max_grad_norm": 0.5,
        },
    },
    "A2C": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            'learning_rate': 0.0007,
            'n_steps': 5,
            'gamma': 0.99,
            'gae_lambda': 1.0,
            'ent_coef':0.01,
            'vf_coef':0.25,
            'max_grad_norm':0.5,
        },
    },
    "DDPG": {
        "total_timesteps": 50000,
        "policy": "MlpPolicy",
        "model_kwargs": {
            "learning_rate": 0.001,
            "buffer_size": 1000000,
            "learning_starts": 100,
            "batch_size": 100,
            "tau": 0.005,
            "gamma": 0.99,
        },
    },
}

In [11]:
trained_models = {}
model_results = {}

for model_name, config in models_to_train.items():
    print(f"\n{'='*50}")
    print(f"Training {model_name} model...")
    print(f"{'='*50}")

    try:
        agent = DRLAgent(env=e_train_gym)

        model = agent.get_model(
            model_name=model_name.lower(),
            policy=config["policy"],
            model_kwargs=config["model_kwargs"],
        )

        trained_model = agent.train_model(
            model=model,
            total_timesteps=config["total_timesteps"],
            tb_log_name=model_name.lower(),
        )

        model_path = f"{TRAINED_MODEL_DIR}/{model_name.lower()}_ff_model"
        trained_model.save(model_path)
        trained_models[model_name] = trained_model

        print(f"{model_name} training completed and saved!")

    except Exception as e:
        print(f"Error training {model_name}: {str(e)}")
        continue

print(f"\nSuccessfully trained {len(trained_models)} models")


Training PPO model...
{'learning_rate': 0.0003, 'n_steps': 2048, 'batch_size': 64, 'n_epochs': 10, 'gamma': 0.99, 'gae_lambda': 0.95, 'clip_range': 0.2, 'vf_coef': 0.5, 'max_grad_norm': 0.5}
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
begin_total_asset:1000000
end_total_asset:2225505.9634471955
Sharpe:  0.7510604712522794
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 1.76e+03  |
|    ep_rew_mean     | 2.58e+09  |
| time/              |           |
|    fps             | 451       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 1211000.5 |
----------------------------------
begin_total_asset:1000000
end_total_asset:2043635.264306779
Sharpe:  0.6701086015466639
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.76

In [16]:
def test_model(model, model_name, env):
    """"
    Test a trained model return results
    """
    print(f'\nTesting {model_name}...')

    obs = env.reset()

    for i in range(len(env.get_attr('df')[0].index.unique())-1):
        action, _states = model.predict(obs)
        obs, rewards, dones, info = env.step(action)
        if dones:
            break

    asset_memory = env.get_attr('asset_memory')[0]
    daily_returns = np.diff(asset_memory) / asset_memory[:-1]
    
    results = {
        'final_value' : asset_memory[-1],
        'total_return' : (asset_memory[-1] / env.get_attr('initial_amount')[0] - 1) * 100,
        'daily_returns' : daily_returns,
        'asset_memory' : asset_memory,
        'date_memory' : env.get_attr('date_memory')[0],
        'actions_memory' : env.get_attr('actions_memory')[0]
    }
    return results

In [ ]:
test_results = {}

for model_name, model in trained_models.items():
    try:
        # Create a vectorized environment for testing
        test_env = DummyVecEnv([lambda: e_trade_gym])

        # Reset test environment
        obs = test_env.reset()

        # Test model
        results = test_model(model, model_name, test_env)
        test_results[model_name] = results

        print(f'{model_name} Results:')
        print(f' Final Portfolio Value: ${results['final_value']:,.2f}')
        print(f' Total Return: {results['total_return']:.2f}%')

        # Calculate additional metrics
        daily_returns = pd.Series(results['daily_returns'])
        sharpe_ratio = daily_returns.mean() / daily_returns.std() * np.sqrt(252)
        max_drawdown = daily_returns.cumsum().min()
    
    except Exception as e:
        print(f'Error testing {model_name}: {str(e)}')
        continue